# <font color='dimgrey'> Topics : Topics Exchange </font>


## <font color='grey'> Till now ... </font>
<font color='grey'>
In the last tutorial 'Direct Exchange' We had producer sending messages via direct exchange pushes to all the queues that has been subscribed to the particular kind of messages in that exchange </font>

<font color='dimgrey'> In this tutorial we will learn  </font>
> [route queues based on multiple criterias or topics](https://www.rabbitmq.com/tutorials/tutorial-five-python.html)

<font color='dimgrey'> <h2> What's different here? </h2></font>

> #### <font color='dimgrey'> We will still be using 'routing_key' and instead of a string we will be taking in a list of strings delimited by dots </font>

> #### <font color='dimgrey'> Binding key matching are done using </font>
<font color='dimgrey'>  1.  Star or '*' to match any one word <br> 2. Hash or '#' to match zero or more words </font>

<font color='dimgrey'>
Suppose we are subscribing to media reviews and it has 3 categories like 1. Type : Movie or TV 2. Genre : Romance, Action, Comedy etc 3. Rating : PG, PG-13, R etc </font>

<font color='dimgrey'>
This would become the topic format : "&lt;type&gt;.&lt;genre&gt;.&lt;rating&gt;" </font>

<font color='dimgrey'> 1. All PG rated comedys : "\*.comedy.pg" <br> 2. All Action tv series : "tv.action.\*" <br> 3. All movies : "movie.\#" <br> 4. Any romance : "\*.romance.\*"</font>

<font color='dimgrey'> <h2> Topic : Superset of fanout and direct </h2></font>

>  <font color='dimgrey'> Use a '\#' as routing_key to make it behave like a fanout </font> <br>
>  <font color='dimgrey'> Don't use a '\#' or a '\*' in the routing_key to make it behave like a direct exchange </font>

<font color='dimgrey'>
Code remains mostly same as the last tutorial. Except for the exchange type in the queue declaration which changes to 'topic' instead of 'direct' </font>

<font color='dimgrey'> <h2> Final Code : Publishing </h2> </font>

In [ ]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            input_ = input("Enter new task s")
            topics, message = input_.split()
            channel.basic_publish(exchange='topic_logs', routing_key=topics, body=message)
            print(" [x] Sent %r:%r" % (topics, message))
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

<font color='dimgrey'> <h2> Final Code : Subscribing </h2> </font>

In [ ]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

severities = input().split()
if not severities:
    sys.stderr.write("Usage: %s [info] [warning] [error]\n" % severities[0])
    sys.exit(1)

for severity in severities:
    channel.queue_bind(
        exchange='topic_logs', queue=queue_name, routing_key=severity)

print(' [*] Waiting for logs. To exit kernel -> Interrupt')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))


channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

try:
    channel.start_consuming()
except KeyboardInterrupt:
    # Exit gracefully
    connection.close()

## <font color='dimgrey'> Testing </font> 
<t><font color='dimgrey'> We will run publisher code from this notebook and we will create two worker nodes for subscribing. We will run the below code after subscriber nodes have declared queues </font> 

In [8]:
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            input_ = input("Enter new task ")
            topics, message = input_.split(' ', 1)
            channel.basic_publish(exchange='topic_logs', routing_key=topics, body=message)
            print(" [x] Sent %r:%r" % (topics, message))
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()
sendQueue()

Enter new tasktv.comedy.pg "a comedy pg 13 tv series"
 [x] Sent 'tv.comedy.pg':'"a comedy pg 13 tv series"'
Enter new taskmovie.romance.r " r rated romance movie"
 [x] Sent 'movie.romance.r':'" r rated romance movie"'


#### <font color='dimgrey'> Will learn about RPC in the next tutorial. Adios!!! </font> 